# Differential Evolution

## Setup

In [1]:
import numpy as np
import toolz
from tqdm import tqdm

from onekit import dekit as dek
from onekit import numpykit as npk
from onekit import optfunckit as ofk
from onekit import pythonkit as pk
from onekit.dekit import (
    Bounds,
    BoundsHandler,
    ObjectiveFunction,
)

In [2]:
def monitor_evolution(de):
    for generation in (
        pbar := tqdm(
            de,
            desc=f"{n_dim}D {func.__name__}",
            bar_format="{desc} | {elapsed}{postfix}",
        )
    ):
        msg = pk.concat_strings(
            ", ",
            f"n_gen={generation.generation_count:_}",
            f"n_pop={generation.population.size:_}",
            f"n_fev={generation.evaluation_count:_}",
            f"n_best={generation.best_so_far_count:_}",
            f"worst={generation.worst.fx:_g}",
            f"best={generation.best.fx:_g}",
        )
        pbar.set_postfix_str(msg)


def print_solution(de, name: str | None = None):
    if name:
        print(pk.headline(f"{name}: {de.message}"))
    else:
        print(pk.headline(de.message))

    solution = de.best
    x_best = denorm(solution.x)
    fx_best = solution.fx
    print(
        pk.concat_strings(
            ", ",
            f"n_gen={de.generation_count:_}",
            f"n_fev={de.evaluation_count:_}",
            f"n_best={de.best_so_far_count:_}",
            f"x_best={x_best}",
            f"fx_best={fx_best:_g}",
        )
    )

## Problem Definition

In [3]:
func = ofk.schwefel
bounds: Bounds = [(-500, 500)] * 4
bnh: BoundsHandler = dek.check_bounds(bounds)
n_dim = bnh.n_dim


def denorm(x: np.ndarray) -> np.ndarray:
    return dek.denormalize(x, x_min=bnh.x_min, x_max=bnh.x_max)


def problem(x: np.ndarray) -> ObjectiveFunction:
    return toolz.pipe(x, denorm, func)

In [4]:
problem(x=np.array([0] * n_dim))

953.5749658744329

## Classic DE/rand/1/bin

In [5]:
rng = npk.check_random_state(seed=101)
max_best_so_far = 30
n_pop = int(25 * np.log(n_dim) * np.sqrt(n_dim))
print(f"{n_pop=} - {max_best_so_far=}")

de_rand_1_bin = dek.DeV1(
    func=problem,
    init_strategy=dek.Initialization.random__standard_uniform(n_pop, n_dim, rng),
    mutation_strategy=dek.Mutation.rand_1(rng),
    bound_repair_strategy=dek.BoundRepair.clip__standard_uniform(),
    crossover_strategy=dek.Crossover.binomial(rng),
    replacement_strategy=dek.Replacement.smaller_is_better(),
    termination_strategy=dek.Termination.has_met_any_basic_strategy(
        max_best_so_far=max_best_so_far,
    ),
    f_strategy=dek.Parameter.dither(0.5, 1.0, rng),
    cr_strategy=dek.Parameter.constant(0.9),
)

monitor_evolution(de_rand_1_bin)

n_pop=69 - max_best_so_far=30


4D schwefel | 00:00, n_gen=213, n_pop=69, n_fev=14_766, n_best=1, worst=5.33358e-05, best=5.09772e-05 


## SHADE 1.1

In [6]:
rng = npk.check_random_state(seed=101)
max_best_so_far = 30
n_pop = int(25 * np.log(n_dim) * np.sqrt(n_dim))
print(f"{n_pop=} - {max_best_so_far=}")

shade11 = dek.DeV4(
    func=problem,
    init_strategy=dek.Initialization.random__standard_uniform(n_pop, n_dim, rng),
    mutation_strategy=dek.Mutation.current_to_pbest_1(rng),
    bound_repair_strategy=dek.BoundRepair.midway(),
    crossover_strategy=dek.Crossover.binomial(rng),
    replacement_strategy=dek.Replacement.smaller_is_better(),
    termination_strategy=dek.Termination.has_met_any_basic_strategy(
        max_best_so_far=max_best_so_far,
    ),
    memory_size=5,
    p_max=0.2,
    seed=rng,
)

monitor_evolution(shade11)

n_pop=69 - max_best_so_far=30


4D schwefel | 00:02, n_gen=149, n_pop=69, n_fev=10_350, n_best=4, worst=5.31081e-05, best=5.09465e-05


## LSHADE

In [7]:
rng = npk.check_random_state(seed=101)
max_fev = 10_000 * n_dim + n_pop
max_best_so_far = 30
n_pop = int(25 * np.log(n_dim) * np.sqrt(n_dim))
print(f"{n_pop=} - {max_fev=} - {max_best_so_far=}")

lshade = dek.DeV5(
    func=problem,
    init_strategy=dek.Initialization.random__standard_uniform(n_pop, n_dim, rng),
    mutation_strategy=dek.Mutation.current_to_pbest_1(rng),
    bound_repair_strategy=dek.BoundRepair.midway(),
    crossover_strategy=dek.Crossover.binomial(rng),
    replacement_strategy=dek.Replacement.smaller_is_better(),
    population_size_adaption_strategy=dek.PopulationSizeAdaption.reduce_population_size_linearly(
        max_fev,
        n_pop_init=n_pop,
        n_pop_min=4,
    ),
    termination_strategy=dek.Termination.has_met_any_basic_strategy(
        max_best_so_far=max_best_so_far,
    ),
    memory_size=5,
    p_max=0.2,
    seed=rng,
)

monitor_evolution(lshade)

n_pop=69 - max_fev=40069 - max_best_so_far=30


4D schwefel | 00:01, n_gen=131, n_pop=56, n_fev=8_205, n_best=1, worst=5.29025e-05, best=5.09641e-05


In [8]:
for name, de in {
    "DE/rand/1/bin": de_rand_1_bin,
    "SHADE": shade11,
    "L-SHADE": lshade,
}.items():
    print_solution(de, name)

-------------------------- DE/rand/1/bin: fx values converged --------------------------
n_gen=213, n_fev=14_766, n_best=1, x_best=[420.96813646 420.9690453  420.96857486 420.96894532], fx_best=5.09772e-05
------------------------------ SHADE: fx values converged ------------------------------
n_gen=149, n_fev=10_350, n_best=4, x_best=[420.96856705 420.96905463 420.96841054 420.96852866], fx_best=5.09465e-05
----------------------------- L-SHADE: fx values converged -----------------------------
n_gen=131, n_fev=8_205, n_best=1, x_best=[420.96898702 420.96914362 420.96894806 420.96915893], fx_best=5.09641e-05
